In [10]:
import pandas as pd

# 파일 경로
input_path = "temp.xlsx"
output_path = "merged_final_from_temp.xlsx"

# 모든 시트 불러오기
xlsx = pd.read_excel(input_path, sheet_name=None)

# 시트 분리
df_work = xlsx["Work"]
df_pub = xlsx["Publication"]
df_coll = xlsx["Collection"]
edge_pw = xlsx["Edge_(Pub-W)"]
edge_pc = xlsx["Edge_(Pub-C)"]

# Step 1: Publication ↔ Work 연결 (Pub-W 엣지 기준)
merged_pw = pd.merge(
    edge_pw,
    df_pub,
    how="left",
    left_on="source id",
    right_on="id",
    suffixes=('', '_pub')
)
merged_pw = pd.merge(
    merged_pw,
    df_work,
    how="left",
    left_on="target id",
    right_on="id",
    suffixes=('', '_work')
)

# Step 2: Publication ↔ Collection 연결 (Pub-C 엣지 기준)
merged_pwc = pd.merge(
    merged_pw,
    edge_pc,
    how="left",
    left_on="source id",
    right_on="source id",
    suffixes=('', '_pc')
)
merged_pwc = pd.merge(
    merged_pwc,
    df_coll,
    how="left",
    left_on="target id_pc",
    right_on="id",
    suffixes=('', '_coll')
)

# Step 3: 필요한 열 선택 및 정리
columns_to_keep = [
    "target id", "name_work", "style",        # Work
    "source id", "institution", "period",     # Publication
    "target id_pc", "name_coll", "author", "type_coll"  # Collection
]

merged_final = merged_pwc[columns_to_keep].copy()

# 컬럼 이름 정제
merged_final.columns = [
    "work_id", "work_title", "work_style",
    "publication_id", "publisher", "published_year",
    "collection_id", "collection_title", "compiler", "collection_type"
]

# Step 4: 엑셀로 저장
with pd.ExcelWriter(output_path, engine='openpyxl', mode='w') as writer:
    merged_final.to_excel(writer, sheet_name="Merged_WPC", index=False)

print(f"✅ 병합 완료: {output_path}")

KeyError: "['name_work'] not in index"

In [4]:
print(merged_wpc.columns.tolist())

['Unnamed: 0', 'source class', 'source id', 'source name', 'target class', 'target id', 'target name', 'relation', 'attribute', '작업자', 'Unnamed: 0_pub', 'id', 'name', 'period', 'type', 'institution', 'provider', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'id_work', 'name_work', 'korname', 'chiname', 'titleExam', 'style', 'original', 'translation', 'url', 'Unnamed: 9_work', 'Unnamed: 0_pc', 'source class_pc', 'source name_pc', 'target class_pc', 'target id_pc', 'target name_pc', 'relation_pc', 'attribute_pc', '작업자_pc', 'Unnamed: 0_coll', 'id_coll', 'name_coll', 'korname_coll', 'chiname_coll', 'author', 'type_coll', 'Unnamed: 7_coll']
